In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
import configparser
import os
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from numpy.linalg import norm
from scipy.spatial.distance import pdist
import plotly_express as px
import seaborn as sns; sns.set()
import configparser
config = configparser.ConfigParser()
config.read("../../../env-sample.ini")
data_home = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data'
output_dir = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/output'
local_lib = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/DS5001-2025-01-R/lessons/lib'
import sys
sys.path.append(local_lib)
from textparser import TextParser

clip_pats = [
    r"\*\*\*\s*START OF",
    r"\*\*\*\s*END OF"
]

# All are 'chap'and 'm'
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
ohco_pat_list = [
    (5200,   rf"^\s*CHAPTER\s+{roman}\s*$"), #Metamorphosis
    (7849,   rf"^\s*{roman}\s*$"), #The Trial
    (6969,  rf"^\s*LETTER .* to .*$"), # The Castle
    (6262,   rf"^CHAPTER\s+{roman}$"), # Amerika
    (6161,   rf"^CHAPTER\s+\d+$"), # The Judgement
    (6060,   rf"^Chapter\s+\d+$"), # Dearest Father
    (6363,  rf"^Chapter\s+\d+$"), # In the Penal colony
    (6464,   rf"^CHAPTER\s+\d+$"), # The Hunger Artist
    (6565, rf"^\s*CHAPTER\s+{roman}\."), # The Jackals and Arabs
    (6666, rf"^\s*CHAPTER\s+{roman}\s*$"), # A Country Doctor
    (6767, rf"^\s*CHAPTER\s+{roman}\s*$"), # An Imperial Message
    (5959,  rf"^(?:ETYMOLOGY|EXTRACTS|CHAPTER)"), # A report for an Academy
    (5858,  rf"^\s*CHAPTER\s+{roman}\.\s*$"), # The Great Wall of China
    (5757, rf"^\s*{roman}\.\s*$"), # The Hunter Gracchus
    (5656,  rf"^\s*{roman}\. .*$"), # Up in the Gallery
    (5555, rf"^CHAPTER\s+{roman}\.?$"), # Before the Law
    (5454, rf"^\s*[A-Z,;-]+\.\s*$"), # Josephine the Songstress
    (5353,  rf"^CHAPTER "), # The Burrow
    (5252, rf"^CHAPTER\s+{roman}\.\s*$"), # Blumfeld
    (23532, rf"Chapter\s+{roman}") # Meditation
]
chapter_regexes = [
    (5200,   rf"^\s*{roman}\s*$"),
    (7849,   rf"^\s*Chapter\s+(?:One|Two|Three|Four|Five|Six|Seven|Eight|Nine|Ten)\s*$"),
    (6969,   rf"^\s*\d+\s*$"),
    (6262,   rf"^\s*\d+\s*$"),
    (6161,   "NOCHAPTERS"),
    (6060,   "NOCHAPTERS"),
    (6363,   "NOCHAPTERS"),
    (6464,   "NOCHAPTERS"),
    (6565,   "NOCHAPTERS"),
    (6666,   "NOCHAPTERS"),
    (6767,   "NOCHAPTERS"),
    (5959,   "NOCHAPTERS"),
    (5858,   "NOCHAPTERS"),
    (5757,   "NOCHAPTERS"),
    (5656,   "NOCHAPTERS"),
    (5555,   "NOCHAPTERS"),
    (5454,   "NOCHAPTERS"),
    (5353,   "NOCHAPTERS"),
    (5252,   "NOCHAPTERS"),
    (23532,  rf"^(Children on the country road|Unmasking a con artist|The Sudden Walk|Resolutions|The trip to the mountains|The Bachelor's Misfortune|The Merchant|Distracted Looking Out|The Way Home|The Passers-by|Passenger|Dresses|The rejection|Food for thought for gentlemen riders|The Alley Window|Desire to become an Indian|The Trees|Unhappiness)$")  # Poem title on line 1
]
ohco_pat_list = chapter_regexes
source_files = f'{data_home}'
source_file_list = sorted(glob(f"{source_files}/*.*"))

book_data = []
for source_file_path in source_file_list:
    # Get the filename only, e.g. 'pg5353.txt'
    filename = os.path.basename(source_file_path)
    # Extract the numeric ID from the filename (remove 'pg' and '.txt')
    book_id = int(filename.replace('pg', '').replace('.txt', ''))
    # Use filename (without extension) as a raw title (optional: clean further)
    book_title = filename.replace('.txt', '').replace('_', ' ')
    # Append a tuple of (book_id, path, title)
    book_data.append((book_id, source_file_path, book_title))
# Convert to DataFrame
LIB = pd.DataFrame(book_data, columns=['book_id', 'source_file_path', 'raw_title']) \
        .set_index('book_id') \
        .sort_index()
book_titles = {
    5200: "Metamorphosis",
    7849: "The Trial",
    6969: "The Castle",
    6262: "Amerika",
    6161: "The Judgement",
    6060: "Dearest Father",
    6363: "In the Penal Colony",
    6464: "The Hunger Artist",
    6565: "The Jackals and Arabs",
    6666: "A Country Doctor",
    6767: "An Imperial Message",
    5959: "A Report for an Academy",
    5858: "The Great Wall of China",
    5757: "The Hunter Gracchus",
    5656: "Up in the Gallery",
    5555: "Before the Law",
    5454: "Josephine the Songstress",
    5353: "The Burrow",
    5252: "Blumfeld",
    23532: "Meditation"
}
book_titles = {f'pg{key}': value for key, value in book_titles.items()}
try:
    LIB['author'] = 'KAFKA, FRANZ'
    LIB['title'] = LIB.raw_title.replace(book_titles).str.upper()
    LIB = LIB.drop('raw_title', axis=1)
except AttributeError:
    pass
LIB['chap_regex'] = LIB.index.map(pd.Series({x[0]:x[1] for x in ohco_pat_list}))
LIB
# This cell takes 16 seconds to run
def tokenize_collection(LIB):

    clip_pats = [
        r"\*\*\*\s*START OF",
        r"\*\*\*\s*END OF"
    ]

    books = []
    for book_id in LIB.index:

        # Announce
        print("Tokenizing", book_id, LIB.loc[book_id].title)

        # Define vars
        chap_regex = LIB.loc[book_id].chap_regex
        ohco_pats = [('chap', chap_regex, 'm')]
        src_file_path = LIB.loc[book_id].source_file_path

        # Create object
        text = TextParser(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats, use_nltk=True)
        # text = TextImporter(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats) 

        # Define parameters
        text.verbose = True
        text.strip_hyphens = True
        text.strip_whitespace = True

        # Parse
        text.import_source().parse_tokens();

        # Name things
        text.TOKENS['book_id'] = book_id
        text.TOKENS = text.TOKENS.reset_index().set_index(['book_id'] + text.OHCO)

        # Add to list
        books.append(text.TOKENS)
        
    # Combine into a single dataframe
    CORPUS = pd.concat(books).sort_index()

    # Clean up
    del(books)
    del(text)
        
    print("Done")
        
    return CORPUS
CORPUS = tokenize_collection(LIB)

CORPUS = CORPUS[CORPUS.term_str != '']
CORPUS['pos_group'] = CORPUS.pos.str[:2]
CORPUS
VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = CORPUS[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['n_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack().count(1)
VOCAB['cat_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos_group.apply(lambda x: set(x))
VOCAB['n_pos'] = CORPUS[['term_str','pos']].value_counts().unstack().count(1)
VOCAB['cat_pos'] = CORPUS[['term_str','pos']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos.apply(lambda x: set(x))
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1
VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['porter_stem'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.apply(lambda x: stemmer2.stem(x.name), 1)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.apply(lambda x: stemmer3.stem(x.name), 1)

VOCAB
# Functions to create TFIDF:
# Bring into your notebook the functions you created previously to generate a BOW table and compute TFIDF values. 
# Extend the TFIDF function so that it also returns the DFIDF value for each term in the VOCAB.
def gather_docs(CORPUS, ohco_level, term_col='term_str'):
    OHCO = CORPUS.index.names
    CORPUS[term_col] = CORPUS[term_col].astype('str')
    DOC = CORPUS.groupby(OHCO[:ohco_level])[term_col].apply(lambda x:' '.join(x)).to_frame('doc_str')
    return DOC

def BOW(tokendf, ocholevel):
    return tokendf.groupby(bags[ocholevel]+['term_str']).term_str.count().to_frame('n') 

def BOWtoTFIDF(BOW, tf_method, CORPUS, ocholevel = 2):
    
    # I added another parameter for the CORPUS because otherwise it becomes really tedious to undo the BOW function

    DTCM = BOW.n.unstack(fill_value=0)
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log2(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    TF = TF.T
    DF = DTCM.astype('bool').sum() 
    N = DTCM.shape[0]
    IDF = IDF = np.log2(N / DF)
    TFIDF = TF * IDF
    # Extend the TFIDF function so that it also returns the DFIDF value for each term in the VOCAB.

    # We can adjust ocho level here if we want to, default is 2
    DOC = gather_docs(CORPUS, 2)
    DOC['n_tokens'] = DOC.doc_str.apply(lambda x: len(x.split()))
    ngram_range = (1,2)
    n_terms = 4000
    count_engine = CountVectorizer(
        stop_words = 'english',
        ngram_range = ngram_range,
        max_features = n_terms)
    X = count_engine.fit_transform(DOC.doc_str)
    DTM = pd.DataFrame(X.toarray(), 
    columns=count_engine.get_feature_names_out(), 
    index=DOC.index)
    VOCAB = DTM.sum().to_frame('n')
    VOCAB.index.name = 'term_str'
    VOCAB['df'] = DTM.astype(bool).sum()
    VOCAB['dfidf'] = VOCAB.df * np.log2(len(DTM)/VOCAB.df)
    VOCAB['dp'] = VOCAB.df / len(DTM)
    VOCAB['di'] = np.log2(1/VOCAB.dp)
    VOCAB['dh'] = VOCAB.dp * VOCAB.di
    VOCAB['n_chars'] = VOCAB.apply(lambda x: len(x.name), 1)
    VOCAB['n_tokens'] = VOCAB.apply(lambda x: len(x.name.split()), 1)
    VOCAB.sort_index()


    #Hopefully this join works but I'm actually not too sure this will work
    
    return TFIDF,VOCAB,DTM

# I think I'm going to use this oneliner more.
# get_tfidf = lambda X, agg_func='sum': (X.T / X.T.agg(agg_func)).T * (np.log2(len(X)/X.astype('bool').sum()))
OHCO = ['book_id','chap_id', 'para_num', 'sent_num', 'token_num']

bags = dict(
    SENTS = OHCO[:4],
    PARAS = OHCO[:3],
    CHAPS = OHCO[:2],
    BOOKS = OHCO[:1]
)
tables = BOWtoTFIDF(BOW(CORPUS,'CHAPS'),'max', CORPUS, 2)
DTM = tables[2]
DFIDF = tables[1]
TFIDF = tables[0]

Tokenizing 5200 METAMORPHOSIS
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5200.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^\s*[IVXLCM]+\s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 5252 BLUMFELD
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5252.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone NOCHAPTERS
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 5353 THE BURROW
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5353.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone NOCHAPTERS
li

c:\Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/DS5001-2025-01-R/lessons/lib\textparser.py:132: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  div_lines = self.TOKENS[src_col].str.contains(div_pat, regex=True, case=True)


Done


## BOW

In [14]:
bow_df= BOW(CORPUS,'CHAPS')

In [12]:
TFIDF.T.columns

MultiIndex([( 5200,  1),
            ( 5200,  2),
            ( 5200,  3),
            ( 5252,  1),
            ( 5353,  1),
            ( 5454,  1),
            ( 5454,  2),
            ( 5555,  1),
            ( 5656,  1),
            ( 5757,  1),
            ( 5858,  1),
            ( 5959,  1),
            ( 6060,  1),
            ( 6161,  1),
            ( 6262,  1),
            ( 6262,  2),
            ( 6262,  3),
            ( 6262,  4),
            ( 6262,  5),
            ( 6262,  6),
            ( 6363,  1),
            ( 6464,  1),
            ( 6565,  1),
            ( 6666,  1),
            ( 6767,  1),
            ( 6969,  1),
            ( 6969,  2),
            ( 6969,  3),
            ( 6969,  4),
            ( 6969,  5),
            ( 6969,  6),
            ( 6969,  7),
            ( 6969,  8),
            ( 6969,  9),
            ( 6969, 10),
            ( 6969, 11),
            ( 6969, 12),
            ( 6969, 13),
            ( 6969, 14),
            ( 6969, 15),


In [ ]:
# Step 1: Reset index so book_id and chap_id become columns
tfidf_reset = TFIDF.reset_index()

# Step 2: Melt all term columns into a long format
tfidf_long = tfidf_reset.melt(
    id_vars=['book_id', 'chap_id'],
    var_name='term_str',
    value_name='tfidf'
)

# Step 3: Merge with bag-of-words dataframe
bow_with_tfidf = bow_df.merge(
    tfidf_long,
    on=['book_id', 'chap_id', 'term_str'],
    how='left'
)

### BOW

In [ ]:
#bow_with_tfidf.to_csv('BOW.csv')

In [33]:
bow_with_tfidf

,book_id,chap_id,term_str,n,tfidf
0,5200,1,a,108,0.000000
1,5200,1,abandoning,1,0.009230
2,5200,1,able,7,0.005868
3,5200,1,about,16,0.008273
4,5200,1,above,2,0.003155
...,...,...,...,...,...
80544,23532,10,you,55,0.068376
80545,23532,10,your,7,0.024125
80546,23532,10,youre,8,0.078501
80547,23532,10,yourself,2,0.018776


## DTM

In [34]:
DTM

term_str         abandoned  abandoning  abilities  ability  able  able make  \
book_id chap_id                                                               
5200    1                0           1          0        0     7          0   
        2                1           1          0        1    13          0   
        3                1           0          0        0     3          0   
5252    1                1           1          0        0     4          0   
5353    1                0           0          1        1     5          0   
...                    ...         ...        ...      ...   ...        ...   
23532   6                0           0          0        0     0          0   
        7                0           0          0        0     0          0   
        8                0           0          0        0     1          0   
        9                0           0          0        0     1          0   
        10               1           0          0        0     1          0   

term_str         abruptly  absence  absent  absolute  ...  ﬁre  ﬁre brigade  \
book_id chap_id                                       ...                     
5200    1               0        0       0         0  ...    0            0   
        2               0        0       0         0  ...    0            0   
        3               1        0       0         0  ...    0            0   
5252    1               0        1       0         0  ...    0            0   
5353    1               0        6       0         0  ...    0            0   
...                   ...      ...     ...       ...  ...  ...          ...   
23532   6               0        0       0         0  ...    0            0   
        7               0        0       0         0  ...    0            0   
        8               0        0       0         0  ...    0            0   
        9               0        0       0         0  ...    0            0   
        10              0        0       0         0  ...    0            0   

term_str         ﬁre engine  ﬁrmly  ﬁrst  ﬁrst time  ﬁst  ﬁt  ﬁxed  ﬂoor  
book_id chap_id                                                           
5200    1                 0      0     0          0    0   0     0     0  
        2                 0      0     0          0    0   0     0     0  
        3                 0      0     0          0    0   0     0     0  
5252    1                 0      0     0          0    0   0     0     0  
5353    1                 0      0     0          0    0   0     0     0  
...                     ...    ...   ...        ...  ...  ..   ...   ...  
23532   6                 0      0     0          0    0   0     0     0  
        7                 0      0     0          0    0   0     0     0  
        8                 0      0     0          0    0   0     0     0  
        9                 0      0     0          0    0   0     0     0  
        10                0      0     0          0    0   0     0     0  

[70 rows x 4000 columns]

In [36]:
#DTM.to_csv('DTM.csv')

## TFIDF

In [37]:
TFIDF

term_str           1   10   15    2   25    3  430  536    6    a  ...  ﬂoors  \
book_id chap_id                                                    ...          
5200    1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
5252    1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
5353    1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...   
23532   6        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        7        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        8        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        9        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
        10       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   

term_str         ﬂopped  ﬂowered  ﬂowers  ﬂown  ﬂung  ﬂushed  ﬂuttered  \
book_id chap_id                                                          
5200    1           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        2           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        3           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
5252    1           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
5353    1           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
...                 ...      ...     ...   ...   ...     ...       ...   
23532   6           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        7           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        8           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        9           0.0      0.0     0.0   0.0   0.0     0.0       0.0   
        10          0.0      0.0     0.0   0.0   0.0     0.0       0.0   

term_str         ﬂuttering  ﬂying  
book_id chap_id                    
5200    1              0.0    0.0  
        2              0.0    0.0  
        3              0.0    0.0  
5252    1              0.0    0.0  
5353    1              0.0    0.0  
...                    ...    ...  
23532   6              0.0    0.0  
        7              0.0    0.0  
        8              0.0    0.0  
        9              0.0    0.0  
        10             0.0    0.0  

[70 rows x 14598 columns]

In [ ]:
#TFIDF.to_csv('TFIDF.csv')

## Reduced and Normalized TFIDF_L2

In [40]:
tfidf_engine = TfidfTransformer(norm='l2', use_idf=True)
X1 = tfidf_engine.fit_transform(DTM)
L2 = pd.DataFrame(X1.toarray(), columns=DTM.columns, index=DTM.index)
L2

term_str         abandoned  abandoning  abilities  ability      able  \
book_id chap_id                                                        
5200    1         0.000000    0.008635   0.000000  0.00000  0.022226   
        2         0.005631    0.007872   0.000000  0.00680  0.037633   
        3         0.005539    0.000000   0.000000  0.00000  0.008543   
5252    1         0.003160    0.004418   0.000000  0.00000  0.006498   
5353    1         0.000000    0.000000   0.004867  0.00438  0.009324   
...                    ...         ...        ...      ...       ...   
23532   6         0.000000    0.000000   0.000000  0.00000  0.000000   
        7         0.000000    0.000000   0.000000  0.00000  0.000000   
        8         0.000000    0.000000   0.000000  0.00000  0.047635   
        9         0.000000    0.000000   0.000000  0.00000  0.091053   
        10        0.035651    0.000000   0.000000  0.00000  0.018329   

term_str         able make  abruptly   absence  absent  absolute  ...  ﬁre  \
book_id chap_id                                                   ...        
5200    1              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        2              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        3              0.0  0.007158  0.000000     0.0       0.0  ...  0.0   
5252    1              0.0  0.000000  0.003593     0.0       0.0  ...  0.0   
5353    1              0.0  0.000000  0.024749     0.0       0.0  ...  0.0   
...                    ...       ...       ...     ...       ...  ...  ...   
23532   6              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        7              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        8              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        9              0.0  0.000000  0.000000     0.0       0.0  ...  0.0   
        10             0.0  0.000000  0.000000     0.0       0.0  ...  0.0   

term_str         ﬁre brigade  ﬁre engine  ﬁrmly  ﬁrst  ﬁrst time  ﬁst   ﬁt  \
book_id chap_id                                                              
5200    1                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        2                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        3                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
5252    1                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
5353    1                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
...                      ...         ...    ...   ...        ...  ...  ...   
23532   6                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        7                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        8                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        9                0.0         0.0    0.0   0.0        0.0  0.0  0.0   
        10               0.0         0.0    0.0   0.0        0.0  0.0  0.0   

term_str         ﬁxed  ﬂoor  
book_id chap_id              
5200    1         0.0   0.0  
        2         0.0   0.0  
        3         0.0   0.0  
5252    1         0.0   0.0  
5353    1         0.0   0.0  
...               ...   ...  
23532   6         0.0   0.0  
        7         0.0   0.0  
        8         0.0   0.0  
        9         0.0   0.0  
        10        0.0   0.0  

[70 rows x 4000 columns]

In [43]:
#L2.to_csv('L2.csv')